# Applied Mathematics for Complex Systems Modeling\n
## Chapter 1: Introduction to Complex Systems Theory\n
### Section 1.3: Distinguishing Complex from Complicated Systems\n
\n
This notebook contains exercises to help understand the fundamental differences between complex and complicated systems. We'll explore how to identify, quantify, and model these different system types.
}
{

## Setup\n
\n
Let's import the libraries we'll need for our exercises:
}
{

In [ ]:
execution_count": null
outputs": []
import numpy as np\n
import matplotlib.pyplot as plt\n
import networkx as nx\n
import pandas as pd\n
from scipy.integrate import solve_ivp\n
from sklearn.decomposition import PCA\n
from scipy import stats\n
\n
# For reproducibility\n
np.random.seed(42)\n
\n
# Set plotting style\n
plt.style.use('seaborn-v0_8-whitegrid')
}
{

## Exercise 1: System Classification (Solved Example)\n
\n
Let's start by understanding the key differences between complex and complicated systems:\n
\n
- **Complicated systems** are typically composed of many parts with well-defined interactions. They can be hard to understand due to their size, but their behavior follows from the sum of their parts. They are often **designed** rather than evolved.\n
\n
- **Complex systems** exhibit emergent behavior that cannot be easily predicted from their components. They adapt, self-organize, and often show non-linear responses to inputs. They typically have **evolved** rather than being designed.\n
\n
Let's create a classification function to help identify systems based on their properties:
}
{

**[Esta celda fue eliminada por errores de formato irreparables. Puedes restaurarla manualmente si lo deseas.]**

### Explanation\n
\n
This exercise demonstrates a quantitative approach to distinguishing between simple, complicated, and complex systems. The key takeaways are:\n
\n
1. **Multi-dimensional classification**: Complex systems are characterized by multiple properties, including nonlinearity, feedback loops, self-organization, and adaptability.\n
\n
2. **Spectrum rather than binary**: Systems exist on a spectrum from simple to complicated to complex, rather than falling into discrete categories.\n
\n
3. **Property weights**: Some properties (like feedback loops and nonlinearity) are more indicative of complexity than others (like number of components).\n
\n
4. **Emergent clusters**: When visualized in the principal component space, systems tend to cluster based on their complexity class.\n
\n
The classification aligned with our intuition: a pendulum is simple, a mechanical watch and computer are complicated, while an ecosystem and stock market are complex.
}
{

## Exercise 2: Linear vs. Nonlinear Response\n
\n
One of the key differences between complicated and complex systems is their response to perturbations. Complicated systems often exhibit linear or predictable responses, while complex systems show nonlinear, sometimes surprising responses due to feedback loops and emergent properties.\n
\n
Let's model and compare the dynamics of a complicated mechanical system versus a complex ecological system under different perturbations.
}
{

In [ ]:
execution_count": null
outputs": []
def complicated_mechanical_system(t, y, params):\n
    \"\"\"Model of a complicated mechanical system (coupled oscillators).\n
    \n
    Parameters:\n
    -----------\n
    t : float\n
        Time\n
    y : array\n
        State variables [x1, v1, x2, v2, ...]\n
    params : dict\n
        System parameters\n
        \n
    Returns:\n
    --------\n
    dydt : array\n
        Derivatives of state variables\n
    \"\"\"\n
    k = params['spring_constant']\n
    m = params['mass']\n
    damping = params['damping']\n
    n_oscillators = len(y) // 2\n
    \n
    dydt = np.zeros_like(y)\n
    \n
    # Update positions (dx/dt = v)\n
    for i in range(n_oscillators):\n
        dydt[2*i] = y[2*i+1]\n
    \n
    # Update velocities (dv/dt = F/m)\n
    for i in range(n_oscillators):\n
        # Spring force from left neighbor\n
        if i > 0:\n
            left_force = k * (y[2*(i-1)] - y[2*i])\n
        else:\n
            left_force = -k * y[2*i]  # Connection to wall\n
        \n
        # Spring force from right neighbor\n
        if i < n_oscillators - 1:\n
            right_force = k * (y[2*(i+1)] - y[2*i])\n
        else:\n
            right_force = 0  # No force if it's the rightmost oscillator\n
        \n
        # Damping force\n
        damping_force = -damping * y[2*i+1]\n
        \n
        # Sum forces and apply F = ma\n
        dydt[2*i+1] = (left_force + right_force + damping_force) / m\n
    \n
    return dydt\n
\n
def complex_ecological_system(t, y, params):\n
    \"\"\"Model of a complex ecological system (food web with predator-prey dynamics).\n
    \n
    Parameters:\n
    -----------\n
    t : float\n
        Time\n
    y : array\n
        Population sizes of different species\n
    params : dict\n
        System parameters\n
        \n
    Returns:\n
    --------\n
    dydt : array\n
        Rate of change of populations\n
    \"\"\"\n
    n_species = len(y)\n
    dydt = np.zeros_like(y)\n
    \n
    # Extract parameters\n
    growth_rates = params['growth_rates']\n
    carrying_capacities = params['carrying_capacities']\n
    interaction_matrix = params['interaction_matrix']\n
    \n
    for i in range(n_species):\n
        # Intrinsic logistic growth\n
        dydt[i] = growth_rates[i] * y[i] * (1 - y[i] / carrying_capacities[i])\n
        \n
        # Species interactions (predation, competition, mutualism)\n
        for j in range(n_species):\n
            if i != j:\n
                dydt[i] += interaction_matrix[i, j] * y[i] * y[j]\n
    \n
    return dydt\n
\n
# Set up the mechanical system (complicated)\n
n_oscillators = 5\n
mechanical_params = {\n
    'spring_constant': 1.0,\n
    'mass': 1.0,\n
    'damping': 0.1\n
}\n
\n
# Initial state: displace first oscillator\n
mechanical_y0 = np.zeros(2 * n_oscillators)\n
mechanical_y0[0] = 1.0  # Displace first oscillator\n
\n
# Set up the ecological system (complex)\n
n_species = 5\n
ecological_params = {\n
    'growth_rates': np.array([0.5, 0.4, 0.3, 0.3, 0.2]),\n
    'carrying_capacities': np.array([100, 80, 60, 70, 50]),\n
    # Interaction matrix: positive = beneficial, negative = harmful\n
    'interaction_matrix': np.array([\n
        [0.0, -0.02, -0.01, 0.0, 0.0],    # Species 0 (prey)\n
        [0.01, 0.0, -0.01, -0.01, 0.0],   # Species 1 (predator of 0, prey of 2,3)\n
        [0.0, 0.005, 0.0, -0.01, -0.01],  # Species 2 (predator of 1, prey of 3,4)\n
        [0.0, 0.005, 0.005, 0.0, -0.01],  # Species 3 (predator of 1,2, prey of 4)\n
        [0.0, 0.0, 0.005, 0.005, 0.0]     # Species 4 (top predator)\n
    ])\n
}\n
\n
# Initial state: all species at half carrying capacity\n
ecological_y0 = ecological_params['carrying_capacities'] / 2\n
\n
# Time points for simulation\n
t_span = (0, 100)\n
t_eval = np.linspace(t_span[0], t_span[1], 1000)\n
\n
# Simulate both systems\n
mechanical_sol = solve_ivp(\n
    lambda t, y: complicated_mechanical_system(t, y, mechanical_params),\n
    t_span, mechanical_y0, t_eval=t_eval, method='RK45'\n
)\n
\n
ecological_sol = solve_ivp(\n
    lambda t, y: complex_ecological_system(t, y, ecological_params),\n
    t_span, ecological_y0, t_eval=t_eval, method='RK45'\n
)\n
\n
# Plot results for both systems\n
fig, axes = plt.subplots(2, 1, figsize=(12, 10))\n
\n
# Plot mechanical system\n
for i in range(n_oscillators):\n
    axes[0].plot(mechanical_sol.t, mechanical_sol.y[2*i], label=f\"Oscillator {i+1}\")\n
axes[0].set_title(\"Complicated System: Coupled Oscillators\")\n
axes[0].set_xlabel(\"Time\")\n
axes[0].set_ylabel(\"Position\")\n
axes[0].legend()\n
axes[0].grid(True)\n
\n
# Plot ecological system\n
for i in range(n_species):\n
    axes[1].plot(ecological_sol.t, ecological_sol.y[i], label=f\"Species {i+1}\")\n
axes[1].set_title(\"Complex System: Ecological Food Web\")\n
axes[1].set_xlabel(\"Time\")\n
axes[1].set_ylabel(\"Population\")\n
axes[1].legend()\n
axes[1].grid(True)\n
\n
plt.tight_layout()\n
plt.show()
}
{

### Your Turn: Testing Perturbation Responses\n
\n
Now, let's explore how these two systems respond to perturbations of different magnitudes. The key difference between complicated and complex systems is that complicated systems typically show proportional responses to perturbations, while complex systems may exhibit threshold effects, regime shifts, or other nonlinear responses.\n
\n
Complete the function below to simulate and analyze the response of both systems to perturbations of varying magnitude.
}
{

**[Esta celda fue eliminada por errores de formato irreparables. Puedes restaurarla manualmente si lo deseas.]**

## Exercise 3: Decomposability Analysis\n
\n
Another key difference between complicated and complex systems is their decomposability. Complicated systems can often be understood by breaking them down into subsystems and analyzing each part separately. Complex systems, on the other hand, lose their essential properties when decomposed because these properties emerge from the interactions between components.\n
\n
Let's explore this concept by creating and analyzing network representations of different systems.
}
{

In [ ]:
execution_count": null
outputs": []
def create_system_network(system_type, n_nodes=20):\n
    \"\"\"Create a network representation of a system.\n
    \n
    Parameters:\n
    -----------\n
    system_type : str\n
        'hierarchical' (for complicated) or 'small-world' (for complex)\n
    n_nodes : int\n
        Number of nodes in the network\n
        \n
    Returns:\n
    --------\n
    G : networkx.Graph\n
        Network representation\n
    \"\"\"\n
    if system_type == 'hierarchical':\n
        # Create a hierarchical network (tree-like structure)\n
        # This represents a complicated system with clear hierarchical organization\n
        G = nx.balanced_tree(2, int(np.log2(n_nodes))-1)  # Binary tree\n
        \n
        # Add some random connections to make it more realistic\n
        for _ in range(int(n_nodes * 0.1)):\n
            u = np.random.randint(0, len(G))\n
            v = np.random.randint(0, len(G))\n
            if u != v and not G.has_edge(u, v):\n
                G.add_edge(u, v)\n
        \n
    elif system_type == 'small-world':\n
        # Create a small-world network\n
        # This represents a complex system with local clustering and global connectivity\n
        k = 4  # Each node connected to k nearest neighbors\n
        p = 0.1  # Probability of rewiring\n
        G = nx.watts_strogatz_graph(n_nodes, k, p)\n
        \n
    else:\n
        raise ValueError(\"Unknown system type\")\n
    \n
    return G\n
\n
def analyze_decomposability(G):\n
    \"\"\"Analyze the decomposability of a network.\n
    \n
    Parameters:\n
    -----------\n
    G : networkx.Graph\n
        Network to analyze\n
        \n
    Returns:\n
    --------\n
    metrics : dict\n
        Dictionary of decomposability metrics\n
    \"\"\"\n
    # Calculate metrics related to decomposability\n
    metrics = {}\n
    \n
    # Modularity\n
    try:\n
        communities = nx.community.greedy_modularity_communities(G)\n
        modularity = nx.community.modularity(G, communities)\n
        metrics['modularity'] = modularity\n
    except:\n
        metrics['modularity'] = np.nan\n
    \n
    # Average clustering coefficient\n
    metrics['clustering_coefficient'] = nx.average_clustering(G)\n
    \n
    # Average shortest path length\n
    try:\n
        metrics['avg_path_length'] = nx.average_shortest_path_length(G)\n
    except:\n
        # Graph might be disconnected\n
        metrics['avg_path_length'] = np.nan\n
    \n
    # Number of connected components\n
    metrics['n_components'] = nx.number_connected_components(G)\n
    \n
    # Diameter\n
    try:\n
        metrics['diameter'] = nx.diameter(G)\n
    except:\n
        metrics['diameter'] = np.nan\n
    \n
    return metrics\n
\n
# Create networks for both system types\n
hierarchical_network = create_system_network('hierarchical', n_nodes=32)\n
small_world_network = create_system_network('small-world', n_nodes=32)\n
\n
# Analyze decomposability\n
hierarchical_metrics = analyze_decomposability(hierarchical_network)\n
small_world_metrics = analyze_decomposability(small_world_network)\n
\n
# Display metrics\n
print(\"Hierarchical Network (Complicated System):\")\n
for metric, value in hierarchical_metrics.items():\n
    print(f\"  {metric}: {value:.4f}\")\n
\n
print(\"\\nSmall-World Network (Complex System):\")\n
for metric, value in small_world_metrics.items():\n
    print(f\"  {metric}: {value:.4f}\")\n
\n
# Visualize the networks\n
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))\n
\n
# Hierarchical network\n
pos1 = nx.spring_layout(hierarchical_network, seed=42)\n
nx.draw(hierarchical_network, pos=pos1, ax=ax1, with_labels=True, \n
        node_color='skyblue', node_size=500, font_size=10)\n
ax1.set_title(\"Hierarchical Network (Complicated System)\")\n
\n
# Small-world network\n
pos2 = nx.circular_layout(small_world_network)\n
nx.draw(small_world_network, pos=pos2, ax=ax2, with_labels=True, \n
        node_color='lightgreen', node_size=500, font_size=10)\n
ax2.set_title(\"Small-World Network (Complex System)\")\n
\n
plt.tight_layout()\n
plt.show()
}
{

### Your Turn: Component Removal Analysis\n
\n
To further explore the difference in decomposability between complicated and complex systems, let's simulate the effect of removing components (nodes) from each network. In a truly decomposable system, removing some components should have localized effects, while in a complex system, even small perturbations can have system-wide impacts.\n
\n
Implement the function below to analyze how network properties change as nodes are removed.
}
{

In [ ]:
execution_count": null
outputs": []
def analyze_node_removal(G, removal_fractions):\n
    \"\"\"Analyze how network properties change as nodes are removed.\n
    \n
    Parameters:\n
    -----------\n
    G : networkx.Graph\n
        Original network\n
    removal_fractions : array\n
        Fractions of nodes to remove\n
        \n
    Returns:\n
    --------\n
    results : dict\n
        Results of the analysis\n
    \"\"\"\n
    # TODO: Implement this function to analyze how network properties change\n
    # as nodes are removed. Suggested metrics to track:\n
    # - Giant component size\n
    # - Average path length\n
    # - Clustering coefficient\n
    # - Connectivity\n
    \n
    # Your code here\n
    \n
    return results\n
\n
# Test with different removal fractions\n
# removal_fractions = np.linspace(0.0, 0.5, 10)\n
# hierarchical_removal_results = analyze_node_removal(hierarchical_network, removal_fractions)\n
# small_world_removal_results = analyze_node_removal(small_world_network, removal_fractions)\n
\n
# Plot the results\n
# fig, axes = plt.subplots(2, 2, figsize=(12, 10))\n
# # Plot results here\n
# plt.tight_layout()\n
# plt.show()
}
{

## Summary\n
{
cells": [
{

# Applied Mathematics for Complex Systems Modeling\n
## Chapter 1: Introduction to Complex Systems Theory\n
### Section 1.3: Distinguishing Complex from Complicated Systems\n
\n
This notebook contains exercises to help understand the fundamental differences between complex and complicated systems. We'll explore how to identify, quantify, and model these different system types.
}
{

## Setup\n
\n
Let's import the libraries we'll need for our exercises:
}
{

In [ ]:
execution_count": null
outputs": []
import numpy as np\n
import matplotlib.pyplot as plt\n
import networkx as nx\n
import pandas as pd\n
from scipy.integrate import solve_ivp\n
from sklearn.decomposition import PCA\n
from scipy import stats\n
\n
# For reproducibility\n
np.random.seed(42)\n
\n
# Set plotting style\n
plt.style.use('seaborn-v0_8-whitegrid')
}
{

## Exercise 1: System Classification (Solved Example)\n
\n
Let's start by understanding the key differences between complex and complicated systems:\n
\n
- **Complicated systems** are typically composed of many parts with well-defined interactions. They can be hard to understand due to their size, but their behavior follows from the sum of their parts. They are often **designed** rather than evolved.\n
\n
- **Complex systems** exhibit emergent behavior that cannot be easily predicted from their components. They adapt, self-organize, and often show non-linear responses to inputs. They typically have **evolved** rather than being designed.\n
\n
Let's create a classification function to help identify systems based on their properties:
}
{